# DeBERTa (Decoding-enhanced BERT with disentangled attention) is a Transformer-based



In [1]:
import pandas as pd
import numpy as np


In [2]:
# conociendo la base de datos
datos_test = pd.read_csv('Valence_test_oc_es.csv', sep=';')
datos_train = pd.read_csv('Valence_train_oc_es.csv', sep=';')
datos_test.head(5)

,ID,Tweet,Dimension,Intensity,Description
0,2018-Es-00766,Dios que susto lpmmm 🙏😡,valence,-2,se infiere un estado emocional moderadamente n...
1,2018-Es-01333,Que lindo que es viajar en moto vieja,valence,2,se infiere un estado emocional moderadamente p...
2,2018-Es-06190,que ansiedad de mierda vo,valence,-3,se infiere un estado emocional muy negativo
3,2018-Es-05144,La ansiedad no desaprovecha ni un segundo.,valence,-2,se infiere un estado emocional moderadamente n...
4,2018-Es-04187,@ts8rr8t Mucho gusto.,valence,2,se infiere un estado emocional moderadamente p...


In [3]:
# Importar las librerías necesarias
import pandas as pd
import torch
from transformers import DebertaTokenizer, DebertaForSequenceClassification
from sklearn.metrics import accuracy_score

In [4]:
# Leer los datos de train y test desde los archivos csv
train_data = pd.read_csv("Valence_train_oc_es.csv", sep=';')
test_data = pd.read_csv("Valence_test_oc_es.csv", sep=';')

In [5]:
# Crear el tokenizador y el modelo de DeBERTa
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=5)

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'pooler.dense.weight', 'classifier.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Convertir los tweets y las etiquetas a tensores
train_tweets = tokenizer(train_data["Tweet"].tolist(), padding=True, truncation=True, return_tensors="pt")
train_labels = torch.tensor(train_data["Intensity"].tolist())
test_tweets = tokenizer(test_data["Tweet"].tolist(), padding=True, truncation=True, return_tensors="pt")
test_labels = torch.tensor(test_data["Intensity"].tolist())


In [7]:
# Definir el optimizador y la función de pérdida
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
# Entrenar el modelo con los datos de train
model.train()
for epoch in range(10):
  outputs = model(**train_tweets, labels=train_labels)
  loss = outputs.loss
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  print(f"Epoch {epoch+1}, Loss: {loss.item()}")

In [ ]:
# Evaluar el modelo con los datos de test
model.eval()
with torch.no_grad():
  outputs = model(**test_tweets)
  predictions = torch.argmax(outputs.logits, dim=1)
  accuracy = accuracy_score(test_labels, predictions)
  print(f"Accuracy: {accuracy}")